In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import torchmetrics as tm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from pathlib import Path
from scipy.io import loadmat
from torchvision.models import resnet50
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.utils import make_grid
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
annotation_train = pd.read_parquet("annotation.parquet")

In [3]:
annotation_train['class'].value_counts()

118    68
78     49
160    48
166    48
143    47
       ..
174    31
63     30
157    29
98     28
135    24
Name: class, Length: 196, dtype: int64

In [4]:
annotation_train = annotation_train[annotation_train['class'].isin([0, 1, 2])]

In [5]:
# Split the data
train_df, valid_df = train_test_split(annotation_train,
                                      test_size=0.2,
                                      random_state=123,
                                      shuffle=True,
                                      stratify=annotation_train['class'])

print(train_df.shape)
print(valid_df.shape)

(96, 2)
(24, 2)


In [6]:
train_df.reset_index(inplace=True)
valid_df.reset_index(inplace=True)
train_df.drop("index", axis=1, inplace=True)
valid_df.drop("index", axis=1, inplace=True)
train_df.head()

,fname,class
0,../../data/cars_train/06784.jpg,2
1,../../data/cars_train/07175.jpg,2
2,../../data/cars_train/02050.jpg,0
3,../../data/cars_train/01750.jpg,2
4,../../data/cars_train/06833.jpg,0


In [7]:
class CarDataset(Dataset):
    def __init__(self, df: pd.DataFrame, transform=None):
        self.img_df = df
        self.transform = transform

    def __len__(self):
        return len(self.img_df)

    def __getitem__(self, item):
        img_path = self.img_df.loc[item, "fname"]
        image = read_image(img_path)
        label = self.img_df.loc[item, "class"]
        if self.transform:
            image = self.transform(image)
        return image, label

In [8]:
class CarClassification(pl.LightningModule):
    def __init__(self, num_of_classes,
                 lr=1e-3, transfer=True,
                 tune_fc=True, hidden_layers=524,
                 optimizer=optim.Adam, batch_size=32):
        super(CarClassification, self).__init__()
        self.resnet = resnet50(pretrained=transfer)
        self.in_features = self.resnet.fc.in_features
        self.optimizer = optimizer
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr
        self.batch_size = batch_size
        if tune_fc:
            for name, param in self.resnet.named_parameters():
                if "bn" not in name:
                    param.requires_grad = False
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.in_features, hidden_layers),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_layers, num_of_classes)
        )

    def forward(self, x):
        return self.resnet(x)

    def configure_optimizers(self):
        return self.optimizer(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomHorizontalFlip(0.3),
            transforms.RandomVerticalFlip(0.3),
            transforms.Lambda(lambda x: x / 255.)
        ])
        img_train = CarDataset(train_df, transform=transform)
        return DataLoader(img_train, batch_size=self.batch_size, shuffle=True)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.long()
        preds = self(x)
        loss = self.criterion(preds, y)
        max_vals, argmax = preds.max(-1)
        accuracy_metrics = tm.functional.accuracy
        accuracy = accuracy_metrics(argmax, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_accuracy", accuracy, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def val_dataloader(self):
        transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.Lambda(lambda x: x / 255.)
        ])
        img_val = CarDataset(valid_df, transform=transform)
        return DataLoader(img_val, batch_size=1, shuffle=False)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.long()
        preds = self(x)
        max_vals, argmax = preds.max(-1)
        loss = self.criterion(preds, y)
        accuracy_metrics = tm.functional.accuracy
        accuracy = accuracy_metrics(argmax, y)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("valid_accuracy", accuracy, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

In [9]:
model = CarClassification(3)

In [10]:
trainer = pl.Trainer(gpus=1, max_epochs=50, progress_bar_refresh_rate=20)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [11]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | resnet    | ResNet           | 24.6 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.1 M     Trainable params
23.5 M    Non-trainable params
24.6 M    Total params
98.333    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/dauren/anaconda3/envs/cars/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/dauren/anaconda3/envs/cars/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/dauren/anaconda3/envs/cars/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning: The number of training samples (3) is smaller than the logging interval Trainer(log_every_n_steps=5

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [12]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/